In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
project_path = '/media/brainstimmaps/DATA/20xx_Projects/2025_DBSinDepression/03_Data'

In [3]:
patients_dir = sorted([el for el in os.listdir(project_path) if el.startswith('Patient')])

In [4]:
till_csv = 'VTA_tracts/plot/outcomes/Depression_scales_evolution.csv'

In [5]:
table_dict = {}
baseline_dict = {}

In [6]:
for pat in patients_dir:

    this_pat= {}    
    complete_path = os.path.join(project_path,pat,till_csv)
    df = pd.read_csv(complete_path) #already sorted in time and with values in percentage

    baseline_df = df[df.date == min(df.date)][['measure','value']]
    baseline_dict[pat] = dict(zip(baseline_df.measure,baseline_df.value ))
    this_pat['n_sessions'] = df.date.nunique()-1 #remove preoperative baseline date
    
    
    df['date'] = pd.to_datetime(df.date,format = '%Y-%m-%d')    
    unique_dates_no_bl = np.array(df['date'].unique()[1:])    
    time_between_assessments = np.diff(df['date'].unique()[1:]) / np.timedelta64(1,'D')
    
    # in days
    this_pat['min'] = np.min(time_between_assessments)
    this_pat['max'] = np.max(time_between_assessments)
    this_pat['std'] = np.round(np.std(time_between_assessments),2)
    this_pat['median'] = np.round(np.median(time_between_assessments),2)
    
    # in months
    this_pat['total_coverage(m)'] = np.ceil((unique_dates_no_bl[-1] - unique_dates_no_bl[0])/ np.timedelta64(1,'D')/30)

#     print('min, max, std, mean time between assessments in days')
#     print(np.min(time_between_assessments),np.max(time_between_assessments),np.std(time_between_assessments),np.mean(time_between_assessments))
#     print('distance in month first and last assessment')
#     print((unique_dates_no_bl[-1] - unique_dates_no_bl[0])/ np.timedelta64(1,'D')/30)
#     print()
    table_dict[pat] = this_pat

In [7]:
table_dict

{'Patient1': {'n_sessions': 36,
  'min': 5.0,
  'max': 126.0,
  'std': 25.93,
  'median': 9.0,
  'total_coverage(m)': 25.0},
 'Patient2': {'n_sessions': 2,
  'min': 173.0,
  'max': 173.0,
  'std': 0.0,
  'median': 173.0,
  'total_coverage(m)': 6.0},
 'Patient3': {'n_sessions': 10,
  'min': 6.0,
  'max': 72.0,
  'std': 20.52,
  'median': 23.0,
  'total_coverage(m)': 9.0}}

In [12]:
sessions_df = pd.DataFrame.from_dict(table_dict).T
sessions_df

,n_sessions,min,max,std,median,total_coverage(m)
Patient1,36.0,5.0,126.0,25.93,9.0,25.0
Patient2,2.0,173.0,173.0,0.00,173.0,6.0
Patient3,10.0,6.0,72.0,20.52,23.0,9.0


In [13]:
sessions_df.to_latex()

'\\begin{tabular}{lrrrrrr}\n\\toprule\n{} &  n\\_sessions &    min &    max &    std &  median &  total\\_coverage(m) \\\\\n\\midrule\nPatient1 &        36.0 &    5.0 &  126.0 &  25.93 &     9.0 &               25.0 \\\\\nPatient2 &         2.0 &  173.0 &  173.0 &   0.00 &   173.0 &                6.0 \\\\\nPatient3 &        10.0 &    6.0 &   72.0 &  20.52 &    23.0 &                9.0 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [10]:
baseline_values = pd.DataFrame.from_dict(baseline_dict)[::-1]
baseline_values


,Patient1,Patient2,Patient3
MADRS,61.67,48.33,48.33
Sheehan,80.00,73.33,NaN
SHAPS,78.57,35.71,57.14
SOFAS,30.00,40.00,30.00
HAMD,43.08,36.92,21.54
BDI,65.08,65.08,49.21
BARS,0.00,0.00,0.00
